In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Concatenate
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM,GRU, Dense, Dropout

from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB

In [ ]:
df=pd.read_csv('Clean_Hotel_Reviews_tfidf_remove_sampled.csv')

In [3]:
df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Reviewer_Score,Tags,days_since_review,lat,lng,Review,Days_stayed,Hotel_location,Cleaned_Review,Review_Class
0,13121,1 rue Geoffroy Marie 9th arr 75009 Paris France,40,12/3/2016,8.6,Hotel Panache,Japan,one staff who can t speak English it was alwa...,37,371,...,10.0,"[' Leisure trip ', ' Solo traveler ', ' Classi...",243 day,48.873254,2.343239,one staff who can t speak English it was alwa...,2.0,Paris France,one staff speak english alway tough commun som...,pos
1,381307,Molenwerf 1 1014 AG Amsterdam Netherlands,914,10/4/2015,8.5,Golden Tulip Amsterdam West,United Kingdom,No Negative,0,7586,...,7.9,"[' Leisure trip ', ' Couple ', ' Standard Doub...",669 day,52.385601,4.847060,The coffee machine in the room,2.0,Amsterdam Netherlands,coffe machin room,pos
2,264042,97 Great Russell Street Bloomsbury Camden Lond...,406,3/6/2017,8.2,Radisson Blu Edwardian Kenilworth,United Kingdom,rooms were lovely but very small Possibly a C...,18,2011,...,9.6,"[' Leisure trip ', ' Group ', ' Standard Doubl...",150 day,51.517972,-0.128049,rooms were lovely but very small Possibly a C...,2.0,United Kingdom,room love small possibl central london standar...,pos
3,294029,Capellans 4 Ciutat Vella 08002 Barcelona Spain,387,2/9/2017,8.9,Hotel Barcelona Catedral,United Kingdom,Would be nice if the pool was heated,9,2695,...,8.3,"[' Business trip ', ' Solo traveler ', ' Doubl...",175 day,41.384829,2.175128,Would be nice if the pool was heated Location...,1.0,Barcelona Spain,would nice pool heat locat great staff super h...,pos
4,11690,1 Kings Cross Road Islington London WC1X 9HX U...,628,3/23/2016,8.1,Crowne Plaza London Kings Cross,United Kingdom,No Negative,0,2312,...,10.0,"[' Leisure trip ', ' Couple ', ' Standard Room...",498 day,51.526385,-0.113604,I had a refurbished room and it was perfect I...,1.0,United Kingdom,refurbish room perfect got room servic everyon...,pos


## Build Naive Bayes by BOW and TFIDF vector as the baseline model 

In [4]:
#Vetorize by bag of words and tfidf 
vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()
X_bow=vectorizer.fit_transform(df['Cleaned_Review'].values)
X_tfidf=tfidf_vectorizer.fit_transform(df['Cleaned_Review'].values) 

X_train_bow, X_test, y_train_bow, y_test = train_test_split(X_bow, df['Review_Class'], test_size=0.2,shuffle=True,random_state=1)
X_train_tfidf, X_test, y_train_tfidf, y_test = train_test_split(X_tfidf, df['Review_Class'], test_size=0.2,shuffle=True,random_state=1)
kf = KFold(n_splits=5,shuffle=True,random_state=123)

In [5]:
kf = KFold(n_splits=5,shuffle=True,random_state=123)

In [6]:

def NB_train_test(X_train,X_val,y_train,y_val):
    mnb = MultinomialNB() 
    mnb.fit(X_train, y_train)
    result=pd.DataFrame({'Model':'NB','train_accu':[mnb.score(X_train,y_train)],'test_accu':[mnb.score(X_val,y_val)]})
    return result

def NB_get_all_fold_result(data_X,data_y):
    all_folds_result=pd.DataFrame([])
    for train_index, val_index in kf.split(data_X):
        X_train_fold, X_test_fold = data_X[train_index], data_X[val_index]
        y_train_fold, y_test_fold = data_y[train_index], data_y[val_index]
        nfold_result=NB_train_test(X_train_fold, X_test_fold, y_train_fold, y_test_fold)
        all_folds_result=all_folds_result.append(nfold_result)
        
    mean_result=pd.DataFrame(all_folds_result.mean()).T
    mean_result.rename({'train_accu':'mean_train_accu','test_accu':'mean_test_accu'},axis=1,inplace=True)
    mean_result['model']="NB"
    return mean_result

NB_bow_meanfold_result=NB_get_all_fold_result(X_train_bow,df['Review_Class'])
NB_tfidf_meanfold_result=NB_get_all_fold_result(X_train_tfidf,df['Review_Class'])

In [23]:
NB_tfidf_meanfold_result

,mean_train_accu,mean_test_accu,model
0,0.562303,0.413938,NB


In [24]:
NB_bow_meanfold_result

,mean_train_accu,mean_test_accu,model
0,0.568457,0.410566,NB


## Load reviews' sequencial tokenid and word2id 

In [12]:
filename = 'ordered_review_tokenid_sampled.pkl'
inputfile = open(filename,'rb')
cleaned_rev_nums=pickle.load(inputfile)
inputfile.close()

word2idfile = open('word2id_sampled.pkl','rb')
word2id=pickle.load(word2idfile)
word2idfile.close()

vocabulary_size=len(word2id)

## Categorical columns encoding and train test split 

In [14]:
def transform_to_onehot(data):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(data)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

def generate_added_features(df):
    nation_label_encoder = LabelEncoder()
    nationality_encoded = nation_label_encoder.fit_transform(df['Reviewer_Nationality'])
    location_label_encoder = LabelEncoder()
    location_encoded = location_label_encoder.fit_transform(df['Hotel_location'])
    added_features=np.concatenate((nationality_encoded.reshape(-1,1),location_encoded.reshape(-1,1),df['Days_stayed'].values.reshape(-1,1)),axis=1)
    return added_features

#sequence padding to the maximum length
X_seq=np.array(cleaned_rev_nums)
X_seq_padded=sequence.pad_sequences(X_seq, maxlen=max_len)
y_seq_onehot=transform_to_onehot(data=df['Review_Class'].values)
#preprocess added features(hotel location, revitaewer nationality and days stayes)
added_features=generate_added_features(df)

#train test split 
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq_padded,y_seq_onehot, test_size=0.2,shuffle=True,random_state=1)
X_train_added, X_test_added, _, _ = train_test_split(added_features,added_features, test_size=0.2,shuffle=False )
print(X_train_seq.shape)
print(X_train_added.shape)

(53380, 368)
(53380, 3)


# GRU model building and training 

In [15]:
#added=True
embedding_size=64
added_shape=added_features.shape[1]

In [25]:
#GRU 
def my_GRU_model(vocabulary_size,embedding_size,input_length):
    tf.keras.backend.clear_session()
    model=Sequential()
    model.add(Embedding(vocabulary_size, embedding_size, input_length=input_length))
    model.add(GRU(50))
    model.add(Dense(3, activation='softmax'))
    return model


def GRU_train_test(X_train,X_val,y_train,y_val,vocabulary_size, embedding_size,input_length):
    #model = my_GRU_model_concat(vocabulary_size, embedding_size,input_length) 
    model = my_GRU_model(vocabulary_size, embedding_size,input_length)
    model.compile(loss='categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])
    history_gru=model.fit(X_train,y_train,batch_size=256,epochs=20)  #,validation_data=(x_val, y_val)
    test_result_gru=model.evaluate(X_val,y_val)
    train_loss_gru,train_accu_gru=history_gru.history['loss'],history_gru.history['accuracy']
    result=pd.DataFrame({'Model':'GRU','train_accu':[np.mean(train_accu_gru)],'test_accu':[test_result_gru[-1]]})
    return result


def GRU_get_all_fold_result(data_X,data_y,vocabulary_size, embedding_size,input_length):
    all_folds_result=pd.DataFrame([])
    for train_index, val_index in kf.split(data_X):
        X_train_fold, X_test_fold = data_X[train_index], data_X[val_index]
        y_train_fold, y_test_fold = data_y[train_index], data_y[val_index]      
        nfold_result = GRU_train_test(X_train_fold,X_test_fold, y_train_fold, y_test_fold,vocabulary_size, embedding_size,input_length)
        all_folds_result=all_folds_result.append(nfold_result)
    mean_result=pd.DataFrame(all_folds_result.mean()).T
    mean_result.rename({'train_accu':'mean_train_accu','test_accu':'mean_test_accu'},axis=1,inplace=True)
    mean_result['model']="GRU"
    return mean_result

GRU_meanfold_result=GRU_get_all_fold_result(X_train_seq,y_train_seq,vocabulary_size, embedding_size,input_length=max_len)


Epoch 1/20
167/167 [==============================] - 74s 441ms/step - loss: 0.8509 - accuracy: 0.5919
Epoch 2/20
167/167 [==============================] - 101s 603ms/step - loss: 0.6776 - accuracy: 0.6928
Epoch 3/20
167/167 [==============================] - 100s 599ms/step - loss: 0.6228 - accuracy: 0.7256
Epoch 4/20
167/167 [==============================] - 93s 554ms/step - loss: 0.5837 - accuracy: 0.7475
Epoch 5/20
167/167 [==============================] - 89s 533ms/step - loss: 0.5487 - accuracy: 0.7652
Epoch 6/20
167/167 [==============================] - 81s 484ms/step - loss: 0.5196 - accuracy: 0.7809
Epoch 7/20
167/167 [==============================] - 80s 477ms/step - loss: 0.4912 - accuracy: 0.7939
Epoch 8/20
167/167 [==============================] - 80s 479ms/step - loss: 0.4670 - accuracy: 0.8049
Epoch 9/20
167/167 [==============================] - 81s 482ms/step - loss: 0.4437 - accuracy: 0.8161
Epoch 10/20
167/167 [==============================] - 81s 485ms/step -

In [17]:
#GRU with added features
def my_GRU_model_concat(vocabulary_size,embedding_size,input_length,added_shape=3):
    tf.keras.backend.clear_session()
    embed_input = keras.Input(
    shape=(None,)) 
    inp_num_data_input = keras.Input(
        shape=(added_shape,))
    
    embed_features = keras.layers.Embedding(vocabulary_size, embedding_size)(embed_input)
    gru_features = keras.layers.GRU(50)(embed_features)
    
    # Merge all available features into a single large vector via concatenation
    x = keras.layers.concatenate([gru_features, inp_num_data_input])
    
    #x_norm=tf.keras.layers.BatchNormalization()(x)
    class_pred = keras.layers.Dense(3, activation='softmax')(x)
    model = keras.Model(
        inputs=[embed_input,inp_num_data_input],
        outputs=class_pred
    )
    return model


def GRU_concat_train_test(X_train,X_val,y_train,y_val,vocabulary_size, embedding_size,input_length):
    model = my_GRU_model_concat(vocabulary_size, embedding_size,input_length) 
    model.compile(loss='categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])
    history_gru=model.fit(X_train,y_train,batch_size=256,epochs=20)  #,validation_data=(x_val, y_val)
    test_result_gru=model.evaluate(X_val,y_val)
    train_loss_gru,train_accu_gru=history_gru.history['loss'],history_gru.history['accuracy']
    result=pd.DataFrame({'Model':'GRU_concat','train_accu':[np.mean(train_accu_gru)],'test_accu':[test_result_gru[-1]]})
    return result


def GRU_concat_get_all_fold_result(data_X,data_y,vocabulary_size, embedding_size,input_length):
    all_folds_result=pd.DataFrame([])
    for train_index, val_index in kf.split(data_X[0]):
        X_train_fold, X_test_fold = data_X[0][train_index], data_X[0][val_index]
        y_train_fold, y_test_fold = data_y[train_index], data_y[val_index]
        X_train_fold_added, X_test_fold_added = data_X[1][train_index], data_X[1][val_index]        
        nfold_result = GRU_concat_train_test([X_train_fold,X_train_fold_added], [X_test_fold,X_test_fold_added], y_train_fold, y_test_fold,vocabulary_size, embedding_size,input_length)
        all_folds_result=all_folds_result.append(nfold_result)
    mean_result=pd.DataFrame(all_folds_result.mean()).T
    mean_result.rename({'train_accu':'mean_train_accu','test_accu':'mean_test_accu'},axis=1,inplace=True)
    mean_result['model']="GRU_concat"
    return mean_result

GRU_added_meanfold_result=GRU_concat_get_all_fold_result([X_train_seq,np.nan_to_num(X_train_added,1)],y_train_seq,vocabulary_size, embedding_size,input_length=max_len)
#GRU_added_meanfold_result.to_csv('GRU_added_meanfold_result.csv',index=False)
GRU_added_meanfold_result

Epoch 1/20
167/167 [==============================] - 63s 378ms/step - loss: 9.0789 - accuracy: 0.3339
Epoch 2/20
167/167 [==============================] - 80s 481ms/step - loss: 2.3720 - accuracy: 0.3536
Epoch 3/20
167/167 [==============================] - 81s 485ms/step - loss: 1.0307 - accuracy: 0.5600
Epoch 4/20
167/167 [==============================] - 89s 532ms/step - loss: 0.7029 - accuracy: 0.6856
Epoch 5/20
167/167 [==============================] - 82s 490ms/step - loss: 0.6453 - accuracy: 0.7163
Epoch 6/20
167/167 [==============================] - 80s 478ms/step - loss: 0.6079 - accuracy: 0.7356
Epoch 7/20
167/167 [==============================] - 87s 522ms/step - loss: 0.5753 - accuracy: 0.7527
Epoch 8/20
167/167 [==============================] - 88s 526ms/step - loss: 0.5445 - accuracy: 0.7703
Epoch 9/20
167/167 [==============================] - 88s 524ms/step - loss: 0.5173 - accuracy: 0.7844
Epoch 10/20
167/167 [==============================] - 87s 522ms/step - l

## Performance evaluation

In [29]:
mean_result_df=pd.concat([NB_bow_meanfold_result,NB_tfidf_meanfold_result,GRU_added_meanfold_result,GRU_meanfold_result])
mean_result_df['Embedding']=['BOW','TFIDF','Trained embedding','Trained embedding']
mean_result_df['add_features']=[False,False,True,False]
mean_result_df.rename({'mean_test_accu':'mean_val_accuracy','mean_train_accu':'mean_train_accuracy'},axis=1)
mean_result_df.to_csv("mean_result_df.csv",index=False)
mean_result_df

,mean_train_accu,mean_test_accu,model,Embedding,add_features
0,0.568457,0.410566,NB,BOW,False
0,0.562303,0.413938,NB,TFIDF,False
0,0.765379,0.626002,GRU_concat,Trained embedding,True
0,0.807474,0.621562,GRU,Trained embedding,False


- Result: As we can see from the result, GRU model using reviews with other user background data (hotel location, revitaewer nationality and days stayes) gives us the best performance using 5 fold cross validation. 